In [1]:
import pandas as pd

In [2]:
import numpy as np


In [3]:
all_transactions = pd.read_csv('all_transactions.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
all_transactions['value'] = all_transactions['value'].astype('float64')

In [5]:
from web3 import Web3, IPCProvider

In [ ]:
w3 = Web3(IPCProvider('/shakespeare/sandipan/Ethereum/.ethereum/geth.ipc'))

In [ ]:
def isAccount(address):
    cs_add = Web3.toChecksumAddress(address)
    if len(w3.eth.getCode(cs_add))==0:
        return True
    return False

In [ ]:
def getBalance(addressList,balanceDict,blockNum):
    addressCS = list(map(lambda x:Web3.toChecksumAddress(x),addressList)) # convert to checksum address
    addressAccount = list(filter(lambda x:isAccount(x),addressCS)) # remove contract addresses
    #balances = list(map(lambda x:w3.eth.getBalance(x,blockNum),addressCS)) # obtain balance
    for i in range(len(addressAccount)):
        address,balance = str(addressAccount[i])[2:].lower(),w3.eth.getBalance(addressAccount[i],blockNum)
        balanceDict[address] = balance

In [ ]:
df_1 = pd.DataFrame.from_dict(getBalance(addressList,blockNum))

In [17]:
from multiprocessing import Process,Manager

In [18]:
manager = Manager()
balanceDict = manager.dict()

In [ ]:
jobs = []
for i in range(4):
    p = Process(target=getBalance, args=(addressList,balanceDict,blockNum))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()